In [1]:
import numpy as np
import torch
import pandas as pd
import networkx as nx
import hypernetx as hnx
import itertools
from TensorMethods import *
from sklearn.cluster import KMeans
from TensorMethods import *
import tensorly as tl

In [8]:
# Generate a random tensor
n_nodes = 10
m = 3
K = 2
dim = [K] * m

# Randomly assign each node to a cluster
clusters = np.random.randint(0,K,n_nodes)

# One-hot encoding of the clusters
one_hot = np.zeros((n_nodes,K))
for i in range(n_nodes):
    one_hot[i,clusters[i]] = 1
one_hot = torch.tensor(one_hot)

P_hat = torch.zeros(dim)
for i in range(K):
    for j in range(i,K):
        for k in range(j,K):
            # Random variable uniformly distributed between 0 and 1
            p = np.random.uniform()
            P_hat[i,j,k] = p
            P_hat[i,k,j] = p
            P_hat[j,i,k] = p
            P_hat[j,k,i] = p
            P_hat[k,i,j] = p
            P_hat[k,j,i] = p

In [9]:
# Print the size of P_hat and one_hot
print(P_hat.size())
print(one_hot.shape)

torch.Size([2, 2, 2])
torch.Size([10, 2])


In [36]:
tl.set_backend('pytorch')
P_hat = P_hat.float()
one_hot = one_hot.float()
Q = tl.tucker_to_tensor((P_hat, [one_hot,one_hot,one_hot]))

In [10]:
P_hat = P_hat.float()
one_hot = one_hot.float()

# one_hot multiplied by P_hat with the first mode of P_hat
res = torch.tensordot(one_hot,P_hat,dims = ([1],[0]))
print (res.size())
res = torch.tensordot(one_hot,res,dims = ([1],[1]))
print (res.size())
res = torch.tensordot(one_hot,res,dims = ([1],[2]))
print (res.size())

torch.Size([10, 2, 2])
torch.Size([10, 10, 2])
torch.Size([10, 10, 10])


In [38]:
# Apply the Tucker decomposition to find the factor matrices
core, factors = tl.decomposition.non_negative_tucker(Q, dim, n_iter_max=10000, init='random', tol=1e-6, random_state=42, verbose=True)
core = core.float()

reconstruction error=0.16803112626075745, variation=0.002236783504486084.
reconstruction error=0.1661359965801239, variation=0.001895129680633545.
reconstruction error=0.16452273726463318, variation=0.0016132593154907227.
reconstruction error=0.16314348578453064, variation=0.001379251480102539.
reconstruction error=0.16195940971374512, variation=0.0011840760707855225.
reconstruction error=0.16093869507312775, variation=0.0010207146406173706.
reconstruction error=0.16005535423755646, variation=0.0008833408355712891.
reconstruction error=0.1592879444360733, variation=0.0007674098014831543.
reconstruction error=0.1586187481880188, variation=0.0006691962480545044.
reconstruction error=0.1580330729484558, variation=0.0005856752395629883.
reconstruction error=0.1575186550617218, variation=0.0005144178867340088.
reconstruction error=0.1570652276277542, variation=0.00045342743396759033.
reconstruction error=0.15666420757770538, variation=0.0004010200500488281.
reconstruction error=0.1563083082

In [39]:
A = tl.tucker_to_tensor((core, factors))

In [40]:
# calculate the norm of the difference between the core tensor and the original tensor
norm = torch.norm(A-Q)
print(norm)

tensor(0.0016)


In [41]:
norm = torch.norm(P_hat-core)
print(norm)

tensor(1.1750)


In [42]:
print(core)

tensor([[[4.1304e-01, 8.8026e-01],
         [8.0119e-01, 5.3569e-01]],

        [[3.5532e-04, 3.8606e-01],
         [2.9201e-01, 6.6973e-01]]])


In [43]:
print(P_hat)

tensor([[[0.3771, 0.7221],
         [0.7221, 0.9162]],

        [[0.7221, 0.9162],
         [0.9162, 0.7771]]])


In [13]:
norm = torch.norm(factors[1]-factors[0])
print(norm)

tensor(18.2588)


In [117]:
# The adjacency tensor must be symmetric, then the factor matrices are same, so we can use the first one
factor = factors[0]

# Apply scale-invariant to the factor matrix
factor = scale_invariant(factor)
R_hat = factor[:, 1:]

# Apply K-mean Clustering to the rows of factor matrix
kmeans = KMeans(n_clusters=K, random_state=0).fit(R_hat)
labels = kmeans.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [118]:
# Calculate the accuracy of the clustering
accuracy = np.sum(labels == clusters) / n_nodes
print(accuracy)

0.1


In [5]:
tl.set_backend('pytorch')

In [23]:
import torch

def higher_order_orthogonal_iteration(tensor, ranks, n_iter_max=1000, tol=1e-10):
    """
    Perform Higher Order Orthogonal Iteration (HOOI) on a 3D tensor.

    Parameters:
    - tensor: Input tensor of size I x J x K.
    - ranks: Tuple of target ranks (r1, r2, r3).
    - n_iter_max: Maximum number of iterations.
    - eps: Convergence criterion.

    Returns:
    - L, R, V factor matrices.
    - core_tensor: Core tensor of the Tucker decomposition.
    """
    # Initialize factor matrices
    I, J, K = tensor.shape
    L = torch.randn(I, ranks[0])
    R = torch.randn(J, ranks[1])
    V = torch.randn(K, ranks[2])

    # Initialize core tensor
    core_tensor = torch.zeros(ranks)

    # Iterate until convergence
    for _ in range(n_iter_max):
        # Update L
        L = tl.tenalg.multi_mode_dot(tensor, [R, V], modes=[1, 2], transpose=True)
        L = torch.linalg.qr(L)[0]

        # Update R
        R = tl.tenalg.multi_mode_dot(tensor, [L, V], modes=[0, 2], transpose=True)
        R = torch.linalg.qr(R)[0]

        # Update V
        V = tl.tenalg.multi_mode_dot(tensor, [L, R], modes=[0, 1], transpose=True)
        V = torch.linalg.qr(V)[0]

        # Update core tensor
        core_tensor = tl.tenalg.multi_mode_dot(tensor, [L, R, V], modes=[0, 1, 2])

        # Compute the approximation error
        error = torch.norm(tensor - core_tensor) / torch.norm(tensor)
        if error < tol:
            break

    return L, R, V, core_tensor

# Assuming we have a tensor 'A' and ranks r1, r2, r3 defined elsewhere.
# The actual tensor A and its dimensions should be defined here.
# For example:
# A = torch.randn(10, 10, 10)
# ranks = (3, 3, 3)
# L, R, V, core_tensor = higher_order_orthogonal_iteration(A, ranks)

# This code would then be executed with A and ranks defined, but for the purpose
# of this example, we won't execute it as we don't have a real tensor A to work with.


In [24]:
# Example tensor of size 10 x 10 x 10
A = torch.randn(10, 10, 10)
# Desired ranks for the decomposition
ranks = (3, 3, 3)
# Perform HOOI
L, R, V, core_tensor = higher_order_orthogonal_iteration(A, ranks)


TypeError: transpose() received an invalid combination of arguments - got (list), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)


In [39]:
R = torch.randn(4, 2)

In [45]:
# Check if R has orthogonal columns
R[:,0] @ R[:,1]


-2.037881221987743e-17

In [3]:
from scipy.stats import ortho_group
R = ortho_group.rvs(4)[:,:2]
R.shape

(4, 2)

In [1]:
import torch

tensor = torch.rand(6, 6, 6)
rank = (2, 2, 2)

In [4]:
I, J, K = tensor.shape
R = ortho_group.rvs(J)[:,:rank[1]]
R = torch.tensor(R, dtype=torch.float32)
V = ortho_group.rvs(K)[:,:rank[2]]
V = torch.tensor(V, dtype=torch.float32)

In [5]:
R.T.shape

torch.Size([2, 6])

In [6]:
import tensorflow as tf

def n_mode_product(x, u, n):
    n = int(n)
    # We need one letter per dimension
    # (maybe you could find a workaround for this limitation)
    if n > 26:
        raise ValueError('n is too large.')
    ind = ''.join(chr(ord('a') + i) for i in range(n))
    exp = f'{ind}K...,JK->{ind}J...'
    result = tf.einsum(exp, x, u)
    return torch.tensor(result.numpy())

In [17]:
def multi_mode_dot(tensor, matrices, modes):
    res = tensor
    for mode, matrix in zip(modes, matrices):
        res = n_mode_product(res, matrix, mode)
    return res

In [8]:
C = n_mode_product(tensor, R.T, 1)


In [21]:
C.shape

torch.Size([6, 2, 2])

In [10]:
type(C)

torch.Tensor

In [11]:
C = n_mode_product(C, V.T, 2)

In [20]:
C = multi_mode_dot(tensor,[R.T, V.T], [1, 2])